In [5]:
#se importan las librerias necesarias
import requests
import json 
import random
import pandas as pd
from hashlib import sha1
from time import time
import sqlite3
from wsgiref.simple_server import make_server
from IPython.display import HTML, display_html, display

def regions_app(url, headers):
    #se realiza la consulta y se guardan en formato JSON	
    result = requests.request("GET", url, headers=headers).json()

    #se crea lista vacia para guardar las regiones existentes
    without_region =[]
    regions = []
    for item in result:
    	#al realizar el ciclo se encontro que se acumulaba un elemento vacio
    	#esto sucede porque Bouvet island (Noruega), Heard island and Mcdonald Island (Australia) 	  no tienen una region definida.
    	#por lo tanto, se les asignara 'N/A'
    	if item['region']=='':
        	item['region'] = 'N/A'
        	without_region.append(item)
    	#se llena la lista regions 
    	if item['region'] not in regions:
        	regions.append(item['region'])
    return regions, without_region

def country_data(regions, without_region):
    reg =[]
    city = [] 
    lang = []
    time_ejec = []
    df = pd.DataFrame(columns=('Region', 'City name', 'Language', 'Time(ms)'), index=None)
    if isinstance(regions, list):
        for op in regions:
            #print(op)
            if isinstance(op, str):
            
                if op != 'N/A':
                    t_i = time()
                    url = "https://restcountries.eu/rest/v2/region/" + op
                    countries = requests.request("GET", url).json()
                    #print(countries)
                    country_select = random.choice(countries)
                
                    for lang_data in country_select['languages']:
                        language_name = sha1(lang_data['name'].encode('utf-8')).hexdigest()
                    reg.append(op)
                    city.append(country_select['name'])
                    lang.append(language_name) 
                    t_f = time()
                    time_ejec.append(float("%.2f" % ((t_f-t_i)*1000)))
                elif op == 'N/A':
                    t_i2 = time()
                    country_select = random.choice(without_region)
                    #bouvet island no tiene un idioma definido, por lo tanto se le asigna N/A
                    if country_select['languages']== []:
                        country_select['languages'] = 'N/A'
                    for lang_data in country_select['languages']:
                        language_name = sha1(lang_data.encode('utf-8')).hexdigest()
                    reg.append(op)
                    city.append(country_select['name'])
                    lang.append(language_name)
                    t_f2 = time()
                    time_ejec.append(float("%.2f" % ((t_f2-t_i2)*1000)))
                    df['Region'] = reg
                    df['City name'] = city
                    df['Language'] = lang
                    df['Time(ms)'] = time_ejec
                    time_max = df['Time(ms)'].max()
                    time_min = df['Time(ms)'].min()
                    time_total = df['Time(ms)'].sum().round(2)
                    time_ave = df['Time(ms)'].mean().round(2)
                    return df, time_max, time_min, time_total, time_ave
                else:
                    print('No es una opcion valida')
            else:
                print('Tipo de dato no valido: {0}. Por favor, ingrese un valor de tipo String.'.format(type(op)))
    else:
        print('ingrese una lista de regiones')
def table_insert(table_data):
    conn = sqlite3.connect('Prueba_Z.db')
    #c = conn.cursor()
    table_data.to_sql(name='Tabla_Regiones', con=conn)
    #recuerde cambiar la direccion en la que se guarda, esto varia dependiendo de cada equipo.
    table_data.to_json(r'/home/nikole/Escritorio/data.json')
    
def html_out(table_data, max_t, min_t, total_t, ave_t):
    display(HTML('<h2 > REGIONS 2019</h2>'))
    display_html(table_data)
    display(HTML('<h2 > DataFrame creation time</h2>'))
    display_html(HTML('Maximum time: ' + str(max_t) + ' ms'))
    display_html(HTML('Minimum time: ' + str(min_t) + ' ms'))
    display_html(HTML('Total time:   ' + str(total_t) + ' ms'))
    display_html(HTML('Average time: ' + str(ave_t) + ' ms'))
    return 'Display succesful'

if __name__ == "__main__":
    #url donde se extraen todas las caracteristicas de cada pais
    url = "https://restcountries-v1.p.rapidapi.com/all"

    #datos necesarios para poder realizar la consulta
    headers = {
        'x-rapidapi-host': "restcountries-v1.p.rapidapi.com",
        'x-rapidapi-key': "f16396a031msh378ea696111f464p1c6474jsn5a574454b661"
        }

    #se realiza la consulta y se guardan en formato JSON
    [regions, without_region] = regions_app(url, headers)
    [table_data, max_t, min_t, total_t, ave_t] = country_data(regions, without_region)
    table_insert(table_data)
    html_out(table_data, max_t, min_t, total_t, ave_t)



,Region,City name,Language,Time(ms)
0,Asia,Myanmar,460ce8e9a08300639b34b7a8264d1eac65d34d77,530.30
1,Europe,France,44389f6a466eaabf5c1db729e369511134e2b03b,432.87
2,Africa,Guinea-Bissau,23882c575954a0789bf02aba9e6dd01f539bc738,412.54
3,Oceania,Tokelau,649df08a448ee3fa90f3746baaf6b0907df42c91,444.12
4,Americas,Chile,8df7f1b361b2af42d36011e00d22c0f9891ec0b0,372.85
5,Polar,Antarctica,74ad93cbebf587ac46ab0ef1e3f24139ddd3f535,403.74
6,N/A,Bouvet Island,6dcd4ce23d88e2ee9568ba546c007c63d9131c1b,0.02


Maximum time: 530.3 ms

Minimum time: 0.02 ms

Total time: 2596.44 ms

Average time: 370.92 ms